In [1]:
import numpy as np

In [2]:
data = np.load("C:/Users/utente/Documents/UNI/MAGISTRALE/tesi/raw_dataset/artifacts/embeddings/features/_mMCmibgP1phBSwnHz_Rf.npy")
data

array([[1.5526636e-04, 2.6376827e-03, 2.5369141e-03, ..., 0.0000000e+00,
        2.9406552e+00, 1.9667070e+00],
       [3.2264477e-04, 7.1638951e-04, 3.9399308e-03, ..., 1.6012350e-01,
        5.2467567e-01, 1.6269317e-01],
       [8.7242843e-05, 1.4230504e-03, 2.4339557e-03, ..., 2.7850032e-01,
        1.3311268e+00, 8.9523906e-01],
       ...,
       [2.9360544e-04, 3.5794699e-03, 2.5879059e-03, ..., 4.2078042e-01,
        2.3752544e+00, 8.0824184e-01],
       [2.0394982e-04, 3.4768302e-03, 1.8608479e-03, ..., 7.3963804e-03,
        7.7315164e-01, 9.3066508e-01],
       [2.8431651e-04, 3.5392034e-03, 2.6300941e-03, ..., 5.4279852e-01,
        1.5380663e+00, 8.0603528e-01]], shape=(16, 1024), dtype=float32)

In [9]:
data16 = np.load("C:/Users/utente/Documents/UNI/MAGISTRALE/tesi/raw_dataset/artifacts/embeddings/features16/_mMCmibgP1phBSwnHz_Rf.npy")
data16

array([[2.2691996e-04, 5.1769712e-03, 3.1769448e-03, ..., 9.2138321e-04,
        1.3321728e+00, 2.1432409e+00],
       [1.9981456e-04, 4.9672364e-03, 3.4876168e-03, ..., 3.8672460e-03,
        2.0005302e+00, 1.3116370e+00],
       [1.7283800e-04, 4.6990430e-03, 2.2577436e-03, ..., 3.3936050e-02,
        1.1589199e+00, 2.0570502e-01],
       ...,
       [2.9510490e-04, 4.8815403e-03, 1.8096700e-03, ..., 8.9722604e-01,
        2.0379062e+00, 1.0373466e+00],
       [2.7851597e-04, 3.9077518e-03, 1.7861015e-03, ..., 5.1880097e-01,
        6.3291526e-01, 5.7521224e-01],
       [2.7646695e-04, 3.0348096e-03, 3.2888220e-03, ..., 6.7082316e-01,
        1.5321329e+00, 7.4083561e-01]], shape=(16, 1024), dtype=float32)

In [11]:
data162 = np.load("C:/Users/utente/Documents/UNI/MAGISTRALE/tesi/raw_dataset/artifacts/embeddings/features8/_mMCmibgP1phBSwnHz_Rf.npy")
data162

array([[2.2691996e-04, 5.1769712e-03, 3.1769448e-03, ..., 9.2138321e-04,
        1.3321728e+00, 2.1432409e+00],
       [1.3885021e-04, 3.8697184e-03, 2.7642383e-03, ..., 8.8366434e-02,
        1.1579511e+00, 5.1016134e-01],
       [2.3268962e-04, 4.8898738e-03, 1.0630805e-03, ..., 5.7965183e-01,
        8.1942642e-01, 7.3961926e-01],
       ...,
       [2.4933886e-04, 4.7420124e-03, 2.6291262e-03, ..., 2.9148856e-01,
        1.3044943e+00, 1.4150017e+00],
       [2.8614863e-04, 5.2455226e-03, 1.7545322e-03, ..., 7.2473979e-01,
        1.9968475e+00, 1.2079141e+00],
       [2.7646695e-04, 3.0348096e-03, 3.2888220e-03, ..., 6.7082316e-01,
        1.5321329e+00, 7.4083561e-01]], shape=(8, 1024), dtype=float32)

In [10]:
import wandb
import torch
from torchmil.models import ABMIL
import torch.nn as nn

criterion = nn.BCEWithLogitsLoss()
best_model = wandb.restore('best_model.pt', run_path="collicellisilvia-personal/ABMIL_new_frames/abmil_model-zo75tha4:v0")

#model = ABMIL(
#    (1024,),
#    256,
#    att_act="relu",
#    criterion=criterion
#)
#
#model.load_state_dict(
#    torch.load("abmil_model-zo75tha4/best_model.pt", map_location="cpu")
#)
#
#model.eval()

CommError: Could not find run <Run collicellisilvia-personal/ABMIL_new_frames/v0 (not found)>

In [7]:
import torchvision.models as models
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def feat_exct_densenet(device, train=True, last_block="denseblock4"):
    feat = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
    feat_dim = feat.classifier.in_features
    feat.classifier = nn.Identity()
    for par in feat.features.parameters():
        par.requires_grad=False

    if train == False: 
        return feat.to(device).eval()
    
    else:
        for name, param in feat.features.named_parameters():
            if last_block in name:
                param.requires_grad = True

        for module in feat.features.modules():
            if isinstance(module, nn.BatchNorm2d):
                module.eval()

        return feat.to(device), feat_dim
      

In [ ]:
#for name, param in feat_ex.features.named_parameters():
#    if param.requires_grad==True:
#        print(name)
#from torchmil.models import ABMIL

In [76]:
feat_ex, feat_dim = feat_exct_densenet(device)
in_shape = (feat_dim,)
att = ABMIL(in_shape, 256, "relu")
model = nn.Sequential(
    feat_ex,
    att
)

In [ ]:

#data[0][0].size()

for bag_id, (bag_images, bag_name, bag_label) in enumerate(inst_bag_loader):
    bag_images.size()
    #bag_images = bag_images.squeeze(0)
    #with torch.no_grad():
    #    feats = feat_ex(bag_images.to(device)) 
        #feats = feats.unsqueeze(0)
        #print(feats.size())
        #bag_dict = {
        #    'X': feats, 
        #    'Y': bag_label
        #}
        #print(bag_dict['X'].shape[1])
        #print(bag_dict['X'][0].size())
        #for key in bag_dict.keys():
        #    print(f"{key}: {bag_dict[key].shape}")
        #a = att(bag_dict)
        #print(a)
    #print(feats.size())

TypeError: 'bool' object is not callable